In [1]:
#mount the Drive. to store model. 
import os

root = "/content/drive/MyDrive/deep learning colab"
os.chdir(root)

In [5]:
assert os.getcwd() == root

In [6]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [7]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11490434/11490434 [==============================] - 0s 0us/step


In [8]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [9]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [11]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 11s - loss: 1.5174 - accuracy: 0.6115 - val_loss: 0.9363 - val_accuracy: 0.8000 - 11s/epoch - 6ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.7346 - accuracy: 0.8284 - val_loss: 0.5837 - val_accuracy: 0.8588 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 4s - loss: 0.5347 - accuracy: 0.8631 - val_loss: 0.4682 - val_accuracy: 0.8820 - 4s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 6s - loss: 0.4555 - accuracy: 0.8781 - val_loss: 0.4116 - val_accuracy: 0.8934 - 6s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 4s - loss: 0.4125 - accuracy: 0.8865 - val_loss: 0.3776 - val_accuracy: 0.8984 - 4s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 5s - loss: 0.3849 - accuracy: 0.8930 - val_loss: 0.3547 - val_accuracy: 0.9022 - 5s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.3651 - accuracy: 0.8983 - val_loss: 0.3377 - val_accuracy: 0.9064 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 4s - loss: 0.3498 - accuracy: 0.9017 - val_loss: 0.3239 - val_accuracy: 0.9098 - 4s/epoch - 3ms/step
Epoch 

In [12]:
model.save("pretrained_mnist_model.h5")

#Transfer Learning

##New Problem statement -
Classify handwritten digits into odd and even

In [14]:
pretrained_mnist_model = tf.keras.models.load_model("pretrained_mnist_model.h5")  
#previously we have a minst dataset to clasify digits between 0 - 9. now it's a bit same. we need to find even and odd number in the data.
#so we can use the previous saved model and use Transfer learning approch.

In [15]:
pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [16]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}") #check layer's trainable or not.

flatten: True
dense: True
leaky_re_lu: True
dense_1: True
leaky_re_lu_1: True
dense_2: True


In [17]:
for layer in pretrained_mnist_model.layers[:-1]: # leaves the last layer unfreezed => trainable for transfer learning. 
  layer.trainable = False # freezing the layers from getting trained
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False


In [18]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False
dense_2: True


In [ ]:
try:
  del new_model #deleting if model is created.
except:
  pass

In [19]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]

# LAYERS = [
#           lower_pretrained_layers,
#           tf.keras.layers.Dense(2, activation="softmax")
# ]

new_model = tf.keras.models.Sequential(lower_pretrained_layers)

new_model.add(
    tf.keras.layers.Dense(2, activation="softmax")
)

In [20]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 202
Non-traina

In [21]:
100 * 2 + 2  #100 layers * 2 shape = 200 + 2 {expecting outcomes} so = 202

202

In [ ]:
266610 - 1010

In [22]:
265600 + 202 #total parms

265802

In [23]:
np.unique(y_train) #to check the unique values in y_train

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [24]:
9%2 #reminder = 1 odd data

1

In [25]:
6%2 #even data = 0

0

In [28]:
np.where(y_train%2 == 0, 1, 0)  #if training data is % 2 give 0, or 1. last 0 is for dividing and making 0

array([0, 0, 1, ..., 0, 1, 1])

In [29]:
y_train[0], y_train[-1]

(7, 8)

In [30]:
def update_even_odd_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx] = np.where(label % 2 == 0, 1, 0)
  return labels   #this function will replace the data by 0 & 1 in y_train

In [31]:
y_train_bin, y_test_bin, y_valid_bin = update_even_odd_labels([y_train, y_test, y_valid])

In [32]:
np.unique(y_train_bin)

array([0, 1])

In [33]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"]) #now compiling new model

In [34]:
history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=2) #training new data

Epoch 1/10
1719/1719 - 5s - loss: 0.3177 - accuracy: 0.8658 - val_loss: 0.2651 - val_accuracy: 0.8940 - 5s/epoch - 3ms/step
Epoch 2/10
1719/1719 - 4s - loss: 0.2781 - accuracy: 0.8870 - val_loss: 0.2467 - val_accuracy: 0.9040 - 4s/epoch - 2ms/step
Epoch 3/10
1719/1719 - 4s - loss: 0.2657 - accuracy: 0.8936 - val_loss: 0.2382 - val_accuracy: 0.9068 - 4s/epoch - 2ms/step
Epoch 4/10
1719/1719 - 4s - loss: 0.2583 - accuracy: 0.8974 - val_loss: 0.2333 - val_accuracy: 0.9108 - 4s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 4s - loss: 0.2524 - accuracy: 0.9003 - val_loss: 0.2265 - val_accuracy: 0.9146 - 4s/epoch - 2ms/step
Epoch 6/10
1719/1719 - 4s - loss: 0.2489 - accuracy: 0.9016 - val_loss: 0.2225 - val_accuracy: 0.9154 - 4s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.2450 - accuracy: 0.9030 - val_loss: 0.2242 - val_accuracy: 0.9154 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 4s - loss: 0.2425 - accuracy: 0.9051 - val_loss: 0.2209 - val_accuracy: 0.9184 - 4s/epoch - 2ms/step
Epoch 9/

In [35]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.2389 - accuracy: 0.9075


[0.23889565467834473, 0.9075000286102295]

In [36]:
X_new = X_test[:3]

y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [37]:
np.argmax(new_model.predict(X_new), axis=-1)  #predicting even or odd

1/1 [==============================] - 0s 90ms/step


array([0, 1, 0])

#TF can be used for both classification and regression